<div class="alert alert-success">
<h1>BlackJack</h1>
<p>
</p>    
    
</div>

In [ ]:
import tkinter as tk
from tkinter import simpledialog
import random

SUITS = ["♠", "♥", "♦", "♣"]
RANKS = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]

def create_deck():
    return [(rank, suit) for suit in SUITS for rank in RANKS]

def card_value(rank):
    if rank in ("J", "Q", "K"):
        return 10
    if rank == "A":
        return 11
    return int(rank)

def hand_value(hand, return_str=False):
    total, aces = 0, 0
    for rank, _ in hand:
        val = card_value(rank)
        total += val
        if rank == "A":
            aces += 1

    values = [total]
    while aces > 0:
        total -= 10
        values.append(total)
        aces -= 1

    valid_values = sorted(set(v for v in values if v <= 21), reverse=True)
    if not valid_values:
        valid_values = [min(values)]

    if return_str:
        if len(values) > 1:
            return " / ".join(str(v) for v in sorted(set(values)))
        return str(values[0])
    return valid_values[0]

class BlackjackGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Blackjack GUI")
        self.root.configure(bg="#006400")

        self.bankroll = 100
        self.bet = 0
        self.last_bet = 0  
        self.deck = []
        self.player_hand = []
        self.dealer_hand = []

        # Mode Split
        self.playing_split = False
        self.split_hands = []
        self.current_hand = 0

        # Control del primer moviment (per bloquejar Double Down després del primer Hit)
        self.first_move_done = False

        # Títol
        self.status_label = tk.Label(root, text="Benvingut al Blackjack!", font=("Arial", 16, "bold"), bg="#006400", fg="white")
        self.status_label.pack(pady=10)

        # Dealer
        self.dealer_frame = tk.Frame(root, bg="#006400")
        self.dealer_frame.pack(pady=10)
        self.dealer_label = tk.Label(self.dealer_frame, text="Dealer", font=("Arial", 14, "bold"), bg="#006400", fg="white")
        self.dealer_label.pack()
        self.dealer_cards = tk.Canvas(self.dealer_frame, width=600, height=140, bg="#006400", highlightthickness=0)
        self.dealer_cards.pack()

        # Jugador
        self.player_frame = tk.Frame(root, bg="#006400")
        self.player_frame.pack(pady=10)
        self.player_label = tk.Label(self.player_frame, text="Jugador", font=("Arial", 14, "bold"), bg="#006400", fg="white")
        self.player_label.pack()
        self.player_cards = tk.Canvas(self.player_frame, width=600, height=200, bg="#006400", highlightthickness=0)
        self.player_cards.pack()

        # Info banc
        self.bank_label = tk.Label(root, text=f"Banca: {self.bankroll} | Aposta: {self.bet}", font=("Arial", 12), bg="#006400", fg="yellow")
        self.bank_label.pack(pady=5)

        # Botons d’accions
        self.buttons_frame = tk.Frame(root, bg="#006400")
        self.buttons_frame.pack(pady=15)

        self.hit_button = tk.Button(self.buttons_frame, text="Hit", width=12, height=2, command=self.hit, bg="white")
        self.hit_button.grid(row=0, column=0, padx=5)

        self.stand_button = tk.Button(self.buttons_frame, text="Stand", width=12, height=2, command=self.stand, bg="white")
        self.stand_button.grid(row=0, column=1, padx=5)

        self.double_button = tk.Button(self.buttons_frame, text="Double Down", width=12, height=2, command=self.double_down, bg="white")
        self.double_button.grid(row=0, column=2, padx=5)

        self.split_button = tk.Button(self.buttons_frame, text="Split", width=12, height=2, command=self.split, bg="white")
        self.split_button.grid(row=0, column=3, padx=5)

        # Botons de nova partida i repetir aposta
        self.bottom_frame = tk.Frame(root, bg="#006400")
        self.bottom_frame.pack(pady=10)

        self.new_button = tk.Button(self.bottom_frame, text="Nova partida", width=20, height=2, command=self.new_round, bg="gold")
        self.new_button.grid(row=0, column=0, padx=5)

        self.repeat_button = tk.Button(self.bottom_frame, text="Repetir aposta", width=20, height=2, command=self.repeat_bet_round, bg="orange")
        self.repeat_button.grid(row=0, column=1, padx=5)

        self.disable_all()
        self.enable_new_game_buttons(first=True)

    def draw_card(self, canvas, x, y, rank, suit, hidden=False):
        w, h = 80, 120
        if hidden:
            canvas.create_rectangle(x, y, x+w, y+h, fill="blue", outline="white", width=2)
            canvas.create_text(x+w/2, y+h/2, text="??", font=("Arial", 20, "bold"), fill="white")
            return
        canvas.create_rectangle(x, y, x+w, y+h, fill="white", outline="black", width=2)
        color = "red" if suit in ("♥", "♦") else "black"
        canvas.create_text(x+10, y+15, text=rank, font=("Arial", 12, "bold"), anchor="nw", fill=color)
        canvas.create_text(x+w-10, y+h-10, text=rank, font=("Arial", 12, "bold"), anchor="se", fill=color)
        canvas.create_text(x+w/2, y+h/2, text=suit, font=("Arial", 28), fill=color)

    def render_split_hands(self, canvas):
        canvas.delete("all")
        for h, hand in enumerate(self.split_hands):
            y_offset = 10 + h * 100
            for i, (rank, suit) in enumerate(hand):
                self.draw_card(canvas, 10 + i*90, y_offset, rank, suit)
            val = hand_value(hand, return_str=True)
            color = "yellow" if h == self.current_hand else "white"
            canvas.create_text(500, y_offset + 50, text=f"Mà {h+1} (valor: {val})", font=("Arial", 12, "bold"), fill=color)

    def render_hand(self, canvas, hand, hide_first=False):
        canvas.delete("all")
        for i, (rank, suit) in enumerate(hand):
            hidden = hide_first and i == 0
            self.draw_card(canvas, 10 + i*90, 10, rank, suit, hidden)

    def start_round(self):
        self.deck = create_deck()
        random.shuffle(self.deck)
        self.player_hand = [self.deck.pop(), self.deck.pop()]
        self.dealer_hand = [self.deck.pop(), self.deck.pop()]
        self.first_move_done = False  # reiniciar
        self.update_display()

        if hand_value(self.player_hand) == 21:
            if hand_value(self.dealer_hand) == 21:
                self.update_display(reveal_dealer=True)
                self.end_round("Empat! Tots dos teniu Blackjack.")
            else:
                self.bankroll += int(self.bet * 1.5)
                self.update_display(reveal_dealer=True)
                self.end_round("Blackjack! 🃏 Pagament 3:2 🎉")
            return

        self.status_label.config(text="Tria una acció...")
        self.enable_valid_moves(first_move=True)
        self.new_button.config(state="disabled")
        self.repeat_button.config(state="disabled")

    def new_round(self):
        if self.bankroll <= 0:
            self.status_label.config(text="No tens diners! Joc acabat.")
            self.disable_all()
            return
        bet = simpledialog.askinteger("Aposta", f"Tens {self.bankroll} €.\nQuina aposta vols fer?", minvalue=1, maxvalue=self.bankroll)
        if bet is None:
            self.status_label.config(text="Has cancel·lat l'aposta.")
            return
        self.bet = bet
        self.last_bet = bet  
        self.start_round()

    def repeat_bet_round(self):
        if self.bankroll <= 0:
            self.status_label.config(text="No tens diners! Joc acabat.")
            self.disable_all()
            return
        if self.last_bet <= 0 or self.last_bet > self.bankroll:
            self.status_label.config(text="No pots repetir l'aposta anterior.")
            return
        self.bet = self.last_bet  
        self.start_round()

    def update_display(self, reveal_dealer=False):
        self.render_hand(self.dealer_cards, self.dealer_hand, hide_first=not reveal_dealer)

        if self.playing_split:
            self.render_split_hands(self.player_cards)
        else:
            self.render_hand(self.player_cards, self.player_hand, hide_first=False)

        dealer_val = hand_value(self.dealer_hand, return_str=True if reveal_dealer else False)
        player_val = hand_value(self.player_hand, return_str=True)
        self.dealer_label.config(text=f"Dealer (valor: {dealer_val if reveal_dealer else '??'})", fg="white", bg="#006400")

        if self.playing_split:
            self.player_label.config(text=f"Jugador (Split: mà {self.current_hand+1})", fg="yellow", bg="#006400")
        else:
            self.player_label.config(text=f"Jugador (valor: {player_val})", fg="white", bg="#006400")

        self.bank_label.config(text=f"Banca: {self.bankroll} | Aposta: {self.bet}")

    def disable_all(self):
        for b in (self.hit_button, self.stand_button, self.double_button, self.split_button):
            b.config(state="disabled")

    def enable_valid_moves(self, first_move=False):
        self.disable_all()
        self.hit_button.config(state="normal")
        self.stand_button.config(state="normal")

        if not self.first_move_done and len(self.player_hand) == 2:
            value = hand_value(self.player_hand)
            ranks = [r for r, _ in self.player_hand]
            if value in (9, 10, 11) or ("A" in ranks and any(r in ("8", "9", "10") for r in ranks)):
                self.double_button.config(state="normal")
            if self.player_hand[0][0] == self.player_hand[1][0]:
                self.split_button.config(state="normal")

    def hit(self):
        self.player_hand.append(self.deck.pop())
        self.first_move_done = True  # després d’un Hit, no es pot fer Double Down
        self.update_display()
        self.enable_valid_moves()
        if hand_value(self.player_hand) > 21:
            self.bankroll -= self.bet
            if self.playing_split:
                self.finish_split_hand()
            else:
                self.end_round("T'has passat! Perds.")

    def stand(self):
        if self.playing_split:
            self.finish_split_hand()
        else:
            self.dealer_turn()
            self.finish_round()

    def double_down(self):
        if self.first_move_done:
            self.status_label.config(text="Només pots fer Double Down al primer moviment.")
            return
        if self.bankroll >= self.bet:
            self.bet *= 2
            self.player_hand.append(self.deck.pop())
            self.first_move_done = True
            self.update_display()
            if hand_value(self.player_hand) > 21:
                self.bankroll -= self.bet
                if self.playing_split:
                    self.finish_split_hand()
                else:
                    self.end_round("T'has passat després del Double Down!")
            else:
                self.stand()
        else:
            self.status_label.config(text="No tens prou diners per fer Double Down.")

    def split(self):
        if self.player_hand[0][0] != self.player_hand[1][0]:
            self.status_label.config(text="Només pots fer Split si tens dues cartes iguals.")
            return
        if self.bankroll < self.bet:
            self.status_label.config(text="No tens prou diners per fer Split.")
            return
        self.bankroll -= self.bet
        self.split_hands = [
            [self.player_hand[0], self.deck.pop()],
            [self.player_hand[1], self.deck.pop()]
        ]
        self.current_hand = 0
        self.playing_split = True
        self.player_hand = self.split_hands[self.current_hand]
        self.first_move_done = False
        self.update_display()
        self.status_label.config(text="Split fet! Jugant la mà 1...")
        self.enable_valid_moves(first_move=True)

    def dealer_turn(self):
        while hand_value(self.dealer_hand) < 17:
            self.dealer_hand.append(self.deck.pop())

    def finish_round(self):
        self.update_display(reveal_dealer=True)
        pv = hand_value(self.player_hand)
        dv = hand_value(self.dealer_hand)
        if pv > 21:
            self.bankroll -= self.bet
            self.end_round("Perds! 😢")
        elif dv > 21 or pv > dv:
            self.bankroll += self.bet
            self.end_round("Has guanyat! 🎉")
        elif pv < dv:
            self.bankroll -= self.bet
            self.end_round("Has perdut! 😢")
        else:
            self.end_round("Empat (push).")

    def finish_split_hand(self):
        self.update_display(reveal_dealer=True)
        pv = hand_value(self.player_hand)
        dv = hand_value(self.dealer_hand)
        msg = ""
        if pv > 21:
            msg = f"Mà {self.current_hand+1}: T'has passat! ❌"
        elif dv > 21 or pv > dv:
            self.bankroll += self.bet
            msg = f"Mà {self.current_hand+1}: Has guanyat! 🎉"
        elif pv < dv:
            self.bankroll -= self.bet
            msg = f"Mà {self.current_hand+1}: Has perdut! 😢"
        else:
            msg = f"Mà {self.current_hand+1}: Empat (push)."
        self.status_label.config(text=msg)
        self.current_hand += 1
        if self.current_hand < len(self.split_hands):
            self.player_hand = self.split_hands[self.current_hand]
            self.first_move_done = False
            self.update_display()
            self.status_label.config(text=f"Jugant la mà {self.current_hand+1}...")
            self.enable_valid_moves(first_move=True)
        else:
            self.playing_split = False
            self.end_round("Split acabat. Resultats mostrats.")

    def end_round(self, message):
        self.update_display(reveal_dealer=True)
        self.status_label.config(text=message)
        self.disable_all()
        self.enable_new_game_buttons()

    def enable_new_game_buttons(self, first=False):
        self.new_button.config(state="normal")
        if not first and self.last_bet > 0:
            self.repeat_button.config(state="normal")
        else:
            self.repeat_button.config(state="disabled")


# --- MAIN ---
if __name__ == "__main__":
    root = tk.Tk()
    game = BlackjackGUI(root)
    root.mainloop()

